# Multiband Models

In this tutorial you will learn how to set up multiband model fits. These are formatted similarly to Group_Model objects, the main difference being that every model has a unique target to fit. That said, one "model" for a multiband model object could itself be a full Group_Model object so in truth there are no limitations to what can be fit across multiple bands! 

It is, of course, more work to set up a fit across multiple wavelength bands. However, the tradeoff can be well worth it in some cases. Perhaps there is space-based data with high resolution, but groundbased data has better S/N. Or perhaps each band individually does not have enough signal for a confident fit, but all three together just might. Perhaps colour information is of paramount importance for a science goal, one would hope that both bands could be treated on equal footing but in a consistent way when extracting profile information. There are a number of reasons why one might wish to try and fit a multiband picture of a galaxy simultaneously. 

When fitting multiple bands one often resorts to forced photometry, somtimes also blurring each image to the same approximate PSF. With AutoProf this is entirely unecessary as one can fit each image in its native PSF simultaneously. The final fits are more meaningful and can encorporate all of the available structure information.

In [ ]:
import autoprof as ap
import numpy as np
import torch
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.stats import iqr

In [ ]:
# First we need some data to work with, let's use LEDA 41136 as our example galaxy

# Our first image is from the DESI Legacy-Survey r-band. This image has a pixelscale of 0.262 arcsec/pixel and is 500 pixels across
target_r = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=187.3119&dec=12.9783&size=500&layer=ls-dr9&pixscale=0.262&bands=r")[0].data, dtype = np.float64),
    pixelscale = 0.262,
    zeropoint = 22.5,
    variance = np.ones((500,500))*0.008**2, # note that the variance is important to ensure all images are compared with proper statistical weight. Here we just use the IQR^2 of the pixel values as the variance, for science data one would use a more accurate variance value
    psf = ap.utils.initialize.gaussian_psf(1.12/2.355, 51, 0.262) # we construct a basic gaussian psf for each image by giving the simga (arcsec), image width (pixels), and pixelscale (arcsec/pixel)
)

# The second image is a unWISE W1 band image. This image has a pixelscale of 2.75 arcsec/pixel and is 52 pixels across
target_W1 = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=187.3119&dec=12.9783&size=52&layer=unwise-neo7&pixscale=2.75&bands=1")[0].data, dtype = np.float64),
    pixelscale = 2.75,
    zeropoint = 25.199,
    variance = np.ones((52,52))*4.9**2,
    psf = ap.utils.initialize.gaussian_psf(6.1/2.355, 21, 2.75),
    origin = (np.array([500,500]))*0.262/2 - (np.array([52,52]))*2.75/2, # here we ensure that the images line up by slightly adjusting the origin
)

# The third image is a GALEX NUV band image. This image has a pixelscale of 1.5 arcsec/pixel and is 90 pixels across
target_NUV = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=187.3119&dec=12.9783&size=90&layer=galex&pixscale=1.5&bands=n")[0].data, dtype = np.float64),
    pixelscale = 1.5,
    zeropoint = 20.08,
    variance = np.ones((90,90))*0.0007**2,
    psf = ap.utils.initialize.gaussian_psf(5.4/2.355, 21, 1.5),
    origin = (np.array([500,500]))*0.262/2 - (np.array([90,90]))*1.5/2,
)

fig1, ax1 = plt.subplots(1, 3, figsize = (18,6))
ap.plots.target_image(fig1, ax1[0], target_r)
ax1[0].set_title("r-band image")
ap.plots.target_image(fig1, ax1[1], target_W1)
ax1[1].set_title("W1-band image")
ap.plots.target_image(fig1, ax1[2], target_NUV)
ax1[2].set_title("NUV-band image")
plt.show()

In [ ]:
# The multiband model will need a target to try and fit, but now that we have multiple images the "target" is
# a Target_Image_List object which points to all three.
target_full = ap.image.Target_Image_List((target_r, target_W1, target_NUV))
# It doesn't really need any other information since everything is already available in the individual targets

In [ ]:
# To make things easy to start, lets just fit a sersic model to all three. In principle one can use arbitrary 
# group models designed for each band individually, but that would be unecessarily complex for a tutorial

model_r = ap.models.AutoProf_Model(
    name = "rband model",
    model_type = "sersic galaxy model",
    target = target_r,
    psf_mode = "full",
)
model_W1 = ap.models.AutoProf_Model(
    name = "W1band model",
    model_type = "sersic galaxy model",
    target = target_W1,
    psf_mode = "full",
)
model_NUV = ap.models.AutoProf_Model(
    name = "NUVband model",
    model_type = "sersic galaxy model",
    target = target_NUV,
    psf_mode = "full",
)

# At this point we would just be fitting three separate models at the same time, not very interesting. Next 
# we add constraints so that some parameters are shared between all the models. It makes sense to fix 
# structure parameters while letting brightness parameters vary between bands so that's what we do here.
model_W1.add_equality_constraint(model_r, ["center", "q", "PA", "n", "Re"])
model_NUV.add_equality_constraint(model_r, ["center", "q", "PA", "n", "Re"])
# Now every model will have a unique Ie, but every other parameter is shared for all three

In [ ]:
# We can now make the multiband model object

model_full = ap.models.Multiband_Model(
    name = "LEDA 41136",
    model_list = [model_r, model_W1, model_NUV],
    target = target_full,
)

model_full.initialize()

In [ ]:
result = ap.fit.LM(model_full, verbose = 1).fit() # here we set a very tight tolerance since the r-band image with 100X more pixels will mostly control the Chi^2, its not necessary but this is good insurance
print(result.message)

In [ ]:
# here we plot the results of the fitting, notice that each band has a different PSF and pixelscale. Also, notice
# that the colour bars represent significantly different ranges since each model was allowed to fit its own Ie.
# meanwhile the center, PA, q, and Re is the same for every model.
fig1, ax1 = plt.subplots(1, 3, figsize = (18,6))
ap.plots.model_image(fig1, ax1, model_full)
ax1[0].set_title("r-band model image")
ax1[1].set_title("W1-band model image")
ax1[2].set_title("NUV-band model image")
plt.show()

In [ ]:
# We can also plot the residual images. As can be seen, the galaxy is fit in all three bands simultaneously
# with the majority of the light removed in all bands. A slight residual can be seen in the W1 and NUV bands.
# This is likely due to the very simplistic gaussian PSF used for the sake of this tutorial and/or a slight 
# missalignment in the images.
fig1, ax1 = plt.subplots(1, 3, figsize = (18,6))
ap.plots.residual_image(fig1, ax1, model_full)
ax1[0].set_title("r-band residual image")
ax1[1].set_title("W1-band residual image")
ax1[2].set_title("NUV-band residual image")
plt.show()

## Group models of multiband models

If you want to analyze more than a single astronomical object, you will need to mix and match group models with multiband models. This can get a bit confusing, since both group models and multiband models are able to hold multiple sub models. To keep it straight always remember that a group model holds many sub models which all have the same target, while a multiband model holds sub models which each have a distinct target. Thus there are two ways to model many astronomical objects across different bands. First, you can create a distinct group model for each band (all sub models have the same target band), then have a multiband model which holds the group models (each group model has a different target band). Second, you can have a distinct multiband model for each astronomical object represented across each band (each submodel is for a different band), then have a group model which holds all of these multiband models (which all have the same "target" which is a Target_Image_List object). The second method is generally prefered as it is easier for the optimizers to work with, and is also easier to keep track of since each multiband model holds all the information for a given astronomical object.

Here we will see an example of a multiband fit of an image which has multiple astronomical objects.

In [ ]:
# First we need some data to work with, let's use a region near the Coma Cluster with a bunch of galaxies

# Our first image is from the DESI Legacy-Survey r-band. This image has a pixelscale of 0.262 arcsec/pixel and is 500 pixels across
target_r = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=195.0588&dec=28.0608&size=1000&layer=ls-dr9&pixscale=0.262&bands=r")[0].data, dtype = np.float64),
    pixelscale = 0.262,
    zeropoint = 22.5,
    variance = np.ones((1000,1000))*0.008**2, # note that the variance is important to ensure all images are compared with proper statistical weight. Here we just use the IQR^2 of the pixel values as the variance, for science data one would use a more accurate variance value
    psf = ap.utils.initialize.gaussian_psf(1.12/2.355, 51, 0.262) # we construct a basic gaussian psf for each image by giving the simga (arcsec), image width (pixels), and pixelscale (arcsec/pixel)
)

# The second image is a unWISE W1 band image. This image has a pixelscale of 2.75 arcsec/pixel and is 52 pixels across
target_W1 = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=195.0588&dec=28.0608&size=104&layer=unwise-neo7&pixscale=2.75&bands=1")[0].data, dtype = np.float64),
    pixelscale = 2.75,
    zeropoint = 25.199,
    variance = np.ones((104,104))*4.9**2,
    psf = ap.utils.initialize.gaussian_psf(6.1/2.355, 21, 2.75),
    origin = (np.array([1000,1000]))*0.262/2 - (np.array([104,104]))*2.75/2, # here we ensure that the images line up by slightly adjusting the origin
)

# The third image is a GALEX NUV band image. This image has a pixelscale of 1.5 arcsec/pixel and is 90 pixels across
target_NUV = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=195.0588&dec=28.0608&size=180&layer=galex&pixscale=1.5&bands=n")[0].data, dtype = np.float64),
    pixelscale = 1.5,
    zeropoint = 20.08,
    variance = np.ones((180,180))*0.0007**2,
    psf = ap.utils.initialize.gaussian_psf(5.4/2.355, 21, 1.5),
    origin = (np.array([1000,1000]))*0.262/2 - (np.array([180,180]))*1.5/2,
)
target_full = ap.image.Target_Image_List((target_r, target_W1, target_NUV))

fig1, ax1 = plt.subplots(1, 3, figsize = (18,6))
ap.plots.target_image(fig1, ax1, target_full)
ax1[0].set_title("r-band image")
ax1[1].set_title("W1-band image")
ax1[2].set_title("NUV-band image")
plt.show()

Next we need to construct models for each galaxy. This is understandably more complex than in the single band case, since now we have three times the amout of data to keep track of. Recall that we will create a number of mutliband models to represent each astronomical object, then put them all together in a larger group model.

In [ ]:
# Here we enter the window parameters by hand, in general one would use a segmentation map or some other automated proceedure to pick out the area for many objects
windows = [
    {"r":[[0,200],[180,430]], "W1": [[0,29],[15,51]], "NUV": [[0,38],[36,76]]},
    {"r":[[275,426],[473,614]], "W1": [[28,48],[47,67]], "NUV": [[48,78],[82,116]]},
    {"r":[[467,665],[617,797]], "W1": [[48,69],[61,81]], "NUV": [[84,115],[111,140]]},
    {"r":[[728,894],[672,827]], "W1": [[71,92],[65,86]], "NUV": [[131,159],[120,150]]},
    {"r":[[380,573],[136,353]], "W1": [[36,62],[14,42]], "NUV": [[66,102],[26,66]]},
]

model_list = []

for i, window in enumerate(windows):
    # create the submodels for this object
    # since PSF convolution for this many models would take a long time, lets use nonparametric models, which is almost as good
    sub_list = []
    sub_list.append(
        ap.models.AutoProf_Model(
            name = f"rband model {i}",
            model_type = "nonparametric galaxy model",
            extend_profile = False, # setting to have the nonparametric models return zero beyond their last fitted radius
            target = target_r,
            window = window["r"],
            psf_mode = "full",
        )
    )
    sub_list.append(
        ap.models.AutoProf_Model(
            name = f"W1band model {i}",
            model_type = "nonparametric galaxy model",
            extend_profile = False,
            target = target_W1,
            window = window["W1"],
            psf_mode = "full",
        )
    )
    sub_list.append(
        ap.models.AutoProf_Model(
            name = f"NUVband model {i}",
            model_type = "nonparametric galaxy model",
            extend_profile = False,
            target = target_NUV,
            window = window["NUV"],
            psf_mode = "full",
        )
    )  
    # ensure equality constraints
    sub_list[1].add_equality_constraint(sub_list[0], ["center", "q", "PA"])
    sub_list[2].add_equality_constraint(sub_list[0], ["center", "q", "PA"])
    # Make the multiband model for this object
    model_list.append(
        ap.models.AutoProf_Model(
            name = f"model {i}",
            model_type = "multiband",
            target = target_full,
            model_list = sub_list,
        )
    )
# Make the full model for this system of objects
MODEL = ap.models.AutoProf_Model(
    name = f"full model",
    model_type = "groupmodel",
    target = target_full,
    model_list = model_list,
)
fig, ax = plt.subplots(1,3, figsize = (16,7))
ap.plots.target_image(fig, ax, MODEL.target)
ap.plots.model_window(fig, ax, MODEL)
ax1[0].set_title("r-band image")
ax1[1].set_title("W1-band image")
ax1[2].set_title("NUV-band image")
plt.show()

In [ ]:
# Now we run the fit, note that this will take a while since we are fitting multiple multiband nonparametric 
# models with PSF convolution. Each of those factors adds time. Note that we fit with Iter, since these model
# windows don't overlap there is no benefit to a full LM fit
MODEL.initialize()
result = ap.fit.Iter(MODEL, method = ap.fit.LM, verbose = 1).fit()
print(result.message)

In [ ]:
# Here we plot the residuals. Again it appears there is a missalignment between the bands, though that's not too
# big of a problem for the sake of a tutorial, it is still something to watch out for in real science cases.
# We can also see in the residuals that there is more than one component to these galaxies, further models could 
# be added to get better fits!
fig, ax = plt.subplots(1, 3, figsize = (18,5))
ap.plots.residual_image(fig, ax, MODEL)
ax1[0].set_title("r-band residual image")
ax1[1].set_title("W1-band residual image")
ax1[2].set_title("NUV-band residual image")
plt.show()

In [ ]:
# here we plot the r-band profiles for reference. We can see that there is a fair amount of structure here
# though the residuals above show clearly that a single nonparametric model is not enough for these galaxies
# further models would need to be added to really get a sense of what's going on for these galaxies.
fig, ax = plt.subplots(figsize = (8,8))
for M in MODEL.model_list:
    ap.plots.galaxy_light_profile(fig, ax, M.model_list[0])
ax.legend()
plt.show()

Note that the last point in several of the profiles turns up. This is due to the very limited modelling being done here. If we had included the neighboring objects then this would not happen. Earlier we set extend_profile = False for the nonparametric models. This way the profile behavior is not much of a problem. Had we left extend_profile = True then AutoProf would assume an exponential profile based on the last two points of the profile and extend infinitely, which is obviously bad if it is increasing.